In [5]:
# Hybrid Recommender System

# Load Dataset


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import os

In [4]:
cd = os.getcwd()
print(os.path.dirname(cd))
data_path = os.path.join(os.path.dirname(cd), 'data')
print(data_path)

c:\Users\trangdo\Documents\HocTap\TrenLop\co_so_tnnt\BTL\ecommerce-website-AI\modelAI
c:\Users\trangdo\Documents\HocTap\TrenLop\co_so_tnnt\BTL\ecommerce-website-AI\modelAI\data


In [5]:
df = pd.read_csv(os.path.join(data_path,'data.csv'))

In [10]:
df.head(5)

,TransactionNo,Items,DateTime,Daypart,DayType,User,Rating,Description,Category
0,1,Bread,2016-10-30 09:58:11,Morning,Weekend,Gloria,3,"A staple food made from flour, water, and yeas...",Pure Veg
1,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend,Louise,5,"Reflecting the flavors of Northern Europe, Sca...",Pure Veg
2,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend,Daniel,5,"Reflecting the flavors of Northern Europe, Sca...",Pure Veg
3,3,Hot chocolate,2016-10-30 10:07:57,Morning,Weekend,Shannon,4,"Though not a cake in the traditional sense, ho...",Beverages
4,3,Jam,2016-10-30 10:07:57,Morning,Weekend,Manuel,3,"A sweet spread made from fruits, commonly used...",Desserts


In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import linear_kernel 


In [74]:
def content_based(item, df):
    new_data = df.drop_duplicates(subset = ['Items']).reset_index(drop=True)

    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer(stop_words='english')

    #Replace NaN with an empty string
    new_data['Description'] = new_data['Description'].fillna('')

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(new_data['Description'])

    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    #Construct a reverse map of indices and movie titles
    indices = pd.Series(new_data.index, index=new_data['Items']).drop_duplicates()

    # Get the index of the movie that matches the title
    idx = indices[item]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = cosine_sim[idx] 

    # Sort the movies based on the similarity scores
    sim_scores = sorted(range(len(sim_scores)), key=lambda i: sim_scores[i], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    items_indices = sim_scores[1:11]
    
    # Return the top 10 most similar movies
    return list(set(new_data['Items'].iloc[items_indices]))


['Cheesecake',
 'ItalianCreamCake',
 'Frittata',
 'Soup',
 'Brioche and salami',
 'Pecan',
 'Chicken sand',
 'Empanadas',
 'Swiss Roll']

In [75]:
def CollaborativeFiltering(item_name, random_user, df):
    user_item_df = df.pivot_table(index=["User"], columns=["Items"], values="Rating")
    item_name1 = user_item_df[item_name]
    random_user_df = user_item_df[user_item_df.index == random_user]
    items_bought = random_user_df.columns[random_user_df.notna().any()].tolist() # items which random user bought
    items_bought_df = user_item_df[items_bought] # only return items bought

    # information on how many items each user bought in total:
    user_item_count = items_bought_df.T.notnull().sum()
                                                             
    user_item_count = user_item_count.reset_index()
    user_item_count.columns = ["User","item_count"] # number of items, which random user bought, were bought by each user , max is random user
    
    # 60% of items bought by random user:
    perc = len(items_bought) * 60 / 100 

    users_same_items = user_item_count[user_item_count["item_count"] > perc]["User"] # only calculate with users who bought more than 60% items together with random user 
    final_df = items_bought_df[items_bought_df.index.isin(users_same_items)]
    
    # caculate corr between each pair users who bought more 60% items together with random user
    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns=["corr"])
    corr_df.index.names = ['user_1', 'user_2']
    corr_df = corr_df.reset_index()

    # Users with a correlation of %30 or more with random user:
    top_users = corr_df[(corr_df["user_1"] == random_user) & (corr_df["corr"] >= 0.3)][
        ["user_2", "corr"]].reset_index(drop=True) # correlation >= 40% with random user 

    top_users = top_users.sort_values(by='corr', ascending=False) # corr between User and random user
    top_users.rename(columns={"user_2": "User"}, inplace=True)

    top_users_ratings = top_users.merge(df[["User", "Items", "Rating"]], how='inner')

    # create a dataframe that insert Items, Rating into top_users
    top_users_ratings = top_users_ratings[top_users_ratings["User"] != random_user]

    top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['Rating'] # a new column weighted_rating = corr * ratings
    top_users_ratings.groupby('Items').agg({"weighted_rating": "mean"}) # caculate weighted ratings which random user can rate for items 

    recommendation_df = top_users_ratings.groupby('Items').agg({"weighted_rating": "mean"})
    recommendation_df = recommendation_df.reset_index()

    # items which random user will like:
    items_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 1].sort_values("weighted_rating", ascending=False)

    item_name_col = user_item_df[item_name]
    moveis_from_item_based = user_item_df.corrwith(item_name_col).sort_values(ascending=False)
    mask = moveis_from_item_based.index != item_name
    moveis_from_item_based = moveis_from_item_based[mask]

    recommend_list = items_to_be_recommend[:10]['Items'].to_list() + moveis_from_item_based[0:10].index.to_list() + moveis_from_item_based[0:10].index.to_list()
    recommend_list = list(set(recommend_list))[:10]
    recommend_list = sorted(recommend_list, key=lambda x: random.random())
    return recommend_list

In [7]:
def item_based(item_name, df):
    user_item_df = df.pivot_table(index=["User"], columns=["Items"], values="Rating")
    item_name_col = user_item_df[item_name]
    moveis_from_item_based = user_item_df.corrwith(item_name_col).sort_values(ascending=False)
    mask = moveis_from_item_based.index != item_name
    moveis_from_item_based = moveis_from_item_based[mask]
    return moveis_from_item_based[0:10].index.to_list()

In [8]:
item_based('Bread', df)

c:\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


['Chocolate crinkles',
 'Butter cookie cake',
 'Brown roll breads',
 'Cupcake',
 'Macaroons',
 'Flan',
 'Drinking chocolate spoons',
 'Lemon and coconut',
 'Baklava',
 'Honey']

In [103]:
def recommend(item, user, df):
  user_item_counts = df.groupby('User')['Items'].nunique()
  
  if (len(user_item_counts) < 10):
    print("Content Based")
    return content_based(item, df)  
  else:
    print("CF")
    return CollaborativeFiltering(item, user, df)

In [113]:
item_name = "Drinking chocolate spoons "
random_user = "Adam"

In [114]:
recommend(item_name, random_user, df)

CF


c:\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


['Salad',
 'Apple pie',
 'Cake',
 'Black-forest-cake',
 'Coffee',
 'Baguette',
 'Brownie',
 'Toast',
 'Bread Pudding',
 'Soup']

In [12]:
df.head(5)

,TransactionNo,Items,DateTime,Daypart,DayType,User,Rating,overview
0,1,Bread,2016-10-30 09:58:11,Morning,Weekend,Gloria,3,"A staple food made from flour, water, and yeas..."
1,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend,Louise,5,"Reflecting the flavors of Northern Europe, Sca..."
2,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend,Daniel,5,"Reflecting the flavors of Northern Europe, Sca..."
3,3,Hot chocolate,2016-10-30 10:07:57,Morning,Weekend,Shannon,4,"Though not a cake in the traditional sense, ho..."
4,3,Jam,2016-10-30 10:07:57,Morning,Weekend,Manuel,3,"A sweet spread made from fruits, commonly used..."


In [13]:
df.shape

(20507, 8)

# Let's Start!

In [14]:
# unique items:
df["Items"].nunique()

93

In [15]:
df["Items"].value_counts().head()

Items
Coffee    5471
Bread     3325
Tea       1435
Cake      1025
Pastry     856
Name: count, dtype: int64

In [16]:
# create the User Item Df:
user_item_df = df.pivot_table(index=["User"], columns=["Items"], values="Rating")
user_item_df.shape

(503, 93)

In [17]:
user_item_df.head(10)

Items,Afternoon with the baker,Alfajores,Almond,Apple pie,Baguette,Baklava,Banana cake,Bare Popcorn,Basket,Bento cake,...,Stollen,Swiss Roll,Tartine,Tea,Tiffin,Toast,Truffles,Vanilla cake,Vegan mincepie,Victorian Sponge
User,,,,,,,,,,,,,,,,,,,,,
Aaron,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.666667,NaN,2.0,NaN,NaN,NaN,NaN
Abigail,NaN,NaN,NaN,1.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,1.666667,NaN,NaN,NaN,NaN,NaN,NaN
Adam,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,3.000000,NaN,2.0,1.0,NaN,NaN,NaN
Adrian,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,1.0,NaN,NaN,NaN,NaN
Agnes,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.500000,NaN,0.0,NaN,NaN,NaN,NaN
Alan,NaN,4.5,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,1.333333,NaN,NaN,NaN,NaN,NaN,NaN
Albert,1.0,NaN,0.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.800000,4.0,NaN,NaN,NaN,NaN,NaN
Alfred,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.500000,2.0,NaN,NaN,NaN,NaN,NaN
Alice,NaN,NaN,NaN,4.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# item-based item recommendation example:
item_name1 = user_item_df[item_name]
user_item_df.corrwith(item_name1).sort_values(ascending=False).head(10) # top 10 items most similarity with item_name

d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Items
Bread                 1.0
Cake                  1.0
Frittata              1.0
Chocolate crinkles    1.0
Pastry                1.0
Soup                  1.0
Tea                   1.0
Coffee                1.0
Hearty & Seasonal    -1.0
Medialuna            -1.0
dtype: float64

In [19]:
item_name1

User
Aaron     NaN
Abigail   NaN
Adam      NaN
Adrian    NaN
Agnes     NaN
           ..
Wilson    NaN
Yolanda   NaN
Yvonne    NaN
Zach      NaN
Zachary   NaN
Name: Chocolate crinkles, Length: 503, dtype: float64

In [20]:
# Let's determine the movies that the user watched.

# Let's choose random user:
# random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

In [21]:
# Let's reduce the #dataset to random user:
random_user_df = user_item_df[user_item_df.index == random_user]
random_user_df

Items,Afternoon with the baker,Alfajores,Almond,Apple pie,Baguette,Baklava,Banana cake,Bare Popcorn,Basket,Bento cake,...,Stollen,Swiss Roll,Tartine,Tea,Tiffin,Toast,Truffles,Vanilla cake,Vegan mincepie,Victorian Sponge
User,,,,,,,,,,,,,,,,,,,,,
Adam,0.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,3.0,NaN,2.0,1.0,NaN,NaN,NaN


In [22]:
# Let's choose non-NaN. Movies watched by all 28491:
items_bought = random_user_df.columns[random_user_df.notna().any()].tolist() # items which random user bought
items_bought

['Afternoon with the baker',
 'Alfajores',
 'Almond',
 'Apple pie',
 'Bread',
 'Brownie',
 'Cake',
 'Coffee',
 'Cookies',
 'Hearty & Seasonal',
 'Hot chocolate',
 'Jam',
 'Juice',
 'Medialuna',
 'Mineral water',
 'Muffin',
 'Pastry',
 'Salad',
 'Sandwich',
 'Soup',
 'Swiss Roll',
 'Tea',
 'Toast',
 'Truffles']

In [23]:
#let's verify:
user_item_df.loc[user_item_df.index == random_user, user_item_df.columns == "Juice"] # return ratings of random user for a item

Items,Juice
User,
Adam,3.0


In [24]:
# How many items have random user bought:
len(items_bought)

24

In [25]:
# we have reduced the dataset based on items watched by random user:
items_bought_df = user_item_df[items_bought] # only return items bought
items_bought_df.head()
# items_bought_df.shape

Items,Afternoon with the baker,Alfajores,Almond,Apple pie,Bread,Brownie,Cake,Coffee,Cookies,Hearty & Seasonal,...,Mineral water,Muffin,Pastry,Salad,Sandwich,Soup,Swiss Roll,Tea,Toast,Truffles
User,,,,,,,,,,,,,,,,,,,,,
Aaron,2.0,NaN,NaN,NaN,2.000000,3.0,2.333333,2.250000,0.0,NaN,...,NaN,1.0,5.0,0.0,4.500000,1.0,NaN,1.666667,2.0,NaN
Abigail,NaN,NaN,NaN,1.0,2.285714,3.5,3.250000,3.578947,5.0,NaN,...,5.0,4.0,1.5,2.0,NaN,5.0,NaN,1.666667,NaN,NaN
Adam,0.0,0.0,0.0,4.0,3.285714,4.0,3.500000,2.916667,0.5,2.0,...,2.0,3.0,0.0,2.0,5.000000,4.0,0.0,3.000000,2.0,1.0
Adrian,NaN,NaN,NaN,4.0,3.285714,0.0,1.000000,2.000000,NaN,NaN,...,4.0,NaN,NaN,NaN,3.666667,1.0,NaN,0.000000,1.0,NaN
Agnes,NaN,NaN,NaN,4.0,4.000000,NaN,1.500000,2.636364,2.0,NaN,...,NaN,4.0,0.0,NaN,2.000000,NaN,NaN,3.500000,0.0,NaN


In [26]:
# information on how many items each user bought in total:
user_item_count = items_bought_df.T.notnull().sum()

user_item_count = user_item_count.reset_index()
user_item_count.columns = ["User","item_count"] # number of items, which random user bought, were bought by each user , max is random user
user_item_count.head()

,User,item_count
0,Aaron,15
1,Abigail,13
2,Adam,24
3,Adrian,10
4,Agnes,13


In [27]:
# 3 user bought 10 items:
user_item_count[user_item_count["item_count"] == 10].count()

User          43
item_count    43
dtype: int64

In [28]:
# 60% of items bought by random user:
perc = len(items_bought) * 60 / 100
perc

14.4

In [29]:
# People who have bought more than 60% items together with random user:
users_same_items = user_item_count[user_item_count["item_count"] > perc]["User"] # only calculate with users who bought more than 60% items together with random user 
users_same_items.head()

0      Aaron
2       Adam
6     Albert
9     Alicia
21       Ann
Name: User, dtype: object

In [30]:
# Let's combine the data of random user and similar users:
final_df = items_bought_df[items_bought_df.index.isin(users_same_items)]
# final_df = items_bought_df[items_bought_df.index.isin(users_same_items)]

# final_df.shape
final_df.T.corr() # corr between two user based on items which random user bought 
# final_df

User,Aaron,Adam,Albert,Alicia,Ann,Annie,Arthur,Austin,Barry,Brandon,...,Sally,Shari,Shelly,Simon,Spencer,Susan,Theresa,Tommy,Vivian,William
User,,,,,,,,,,,,,,,,,,,,,
Aaron,1.000000,0.076119,0.462425,0.142444,-0.238939,-0.330538,0.349338,-0.228658,0.508999,-0.228301,...,0.627303,0.039038,0.599828,0.109074,0.042534,-0.157952,-0.176667,0.121871,-0.358289,0.240899
Adam,0.076119,1.000000,-0.084226,-0.315293,0.280761,0.074519,-0.017725,0.440950,0.494416,0.442111,...,-0.161645,0.490510,-0.353699,0.037743,-0.446293,-0.108726,0.363922,-0.045312,0.021381,0.388837
Albert,0.462425,-0.084226,1.000000,0.692633,-0.490877,-0.318284,0.236616,-0.176322,0.171141,0.077527,...,-0.074273,-0.501821,-0.069378,0.658611,0.181290,0.238446,-0.085097,0.399037,-0.294959,-0.231621
Alicia,0.142444,-0.315293,0.692633,1.000000,-0.227059,-0.557927,-0.006910,0.248624,-0.155664,-0.313249,...,-0.026003,-0.080334,-0.127168,-0.179007,0.150729,-0.212103,-0.316410,0.239822,-0.431651,-0.125598
Ann,-0.238939,0.280761,-0.490877,-0.227059,1.000000,0.152029,-0.524517,0.555554,0.082296,0.234783,...,0.019257,0.163872,-0.328440,-0.436271,-0.227154,-0.406169,0.252304,-0.466186,0.138426,0.439804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Susan,-0.157952,-0.108726,0.238446,-0.212103,-0.406169,0.081498,0.088241,-0.366907,-0.354371,0.011817,...,-0.288240,-0.178032,0.266730,0.682928,0.106286,1.000000,-0.016662,0.275807,0.137707,-0.171942
Theresa,-0.176667,0.363922,-0.085097,-0.316410,0.252304,0.585470,-0.418828,0.362673,0.269192,0.265566,...,0.145168,0.103357,-0.289309,-0.122327,-0.117522,-0.016662,1.000000,-0.248531,0.367295,0.276169
Tommy,0.121871,-0.045312,0.399037,0.239822,-0.466186,-0.315614,0.042556,-0.394884,-0.023106,-0.006015,...,-0.039961,-0.623745,0.039669,0.381354,0.480337,0.275807,-0.248531,1.000000,-0.017506,-0.511199


In [31]:
#corr for all users:
# caculate corr between each pair users who bought more 60% items together with random user
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_1', 'user_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_1,user_2,corr
0,Henry,Louis,-0.884492
1,Janet,Connie,-0.871703
2,Brandon,Gene,-0.862302
3,Aaron,Kay,-0.850448
4,Shari,Joan,-0.834144


In [32]:
# Users with a correlation of %30 or more with random user:
top_users = corr_df[(corr_df["user_1"] == random_user) & (corr_df["corr"] >= 0.3)][
    ["user_2", "corr"]].reset_index(drop=True) # correlation >= 40% with random user 

top_users = top_users.sort_values(by='corr', ascending=False) # corr between User and random user
top_users.rename(columns={"user_2": "User"}, inplace=True)
top_users

,User,corr
7,Gloria,0.533943
6,Brandon,0.442111
5,Janet,0.414107
4,Gretchen,0.391394
3,William,0.388837
2,Darlene,0.387732
1,Ronald,0.382465
0,Jose,0.314199


In [33]:
# let's see the ratings of users:
# create a dataframe that insert Items, Rating into top_users
top_users_ratings = top_users.merge(df[["User", "Items", "Rating"]], how='inner')

top_users_ratings = top_users_ratings[top_users_ratings["User"] != random_user]
top_users_ratings.head()

,User,corr,Items,Rating
0,Gloria,0.533943,Bread,3
1,Gloria,0.533943,Coffee,3
2,Gloria,0.533943,Juice,2
3,Gloria,0.533943,Medialuna,3
4,Gloria,0.533943,Coffee,5


In [34]:
# Calculate the Weighted Average Recommendation Score and keep the first 5 items.

#Let's do a single score with the most similar by corr * rating:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['Rating'] # a new column weighted_rating = corr * ratings
top_users_ratings.groupby('Items').agg({"weighted_rating": "mean"}) # caculate weighted ratings which random user can rate for items 

recommendation_df = top_users_ratings.groupby('Items').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df

,Items,weighted_rating
0,Afternoon with the baker,0.414107
1,Alfajores,0.715663
2,Almond,0.395154
3,Apple pie,1.944184
4,Black-forest-cake,2.135773
5,Bread,1.087634
6,Bread Pudding,1.256797
7,Brownie,1.334636
8,Cake,1.038639
9,Cheesecake,1.174183


In [35]:
# weighted rating greater than 4:
recommendation_df[recommendation_df["weighted_rating"] > 1]

# items which random user will like:
items_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 1].sort_values("weighted_rating", ascending=False)

# items_to_be_recommend.merge(movie[["Items", "title"]])

# #Let's see the top 5 movies:
items_to_be_recommend[:5]

,Items,weighted_rating
4,Black-forest-cake,2.135773
3,Apple pie,1.944184
32,Soup,1.589680
15,Crisps,1.565577
27,Salad,1.542357


In [36]:
# Make an item-based suggestion based on the name of the movie that the user has watched with the highest score.

# ▪ 5 suggestions user-based
# ▪ 5 suggestions item-based
?

# The last highly-rated movie by user 108170:

# user = "Alice"
# item = df[(df["User"] == user) & (df["Rating"] == 5.0)].sort_values(by="DateTime", ascending=False)["Items"][0:1].values[0]
# print(item)


IPython -- An enhanced Interactive Python

IPython offers a fully compatible replacement for the standard Python
interpreter, with convenient shell features, special commands, command
history mechanism and output results caching.

At your system command line, type 'ipython -h' to see the command line
options available. This document only describes interactive features.

GETTING HELP
------------

Within IPython you have various way to access help:

  ?         -> Introduction and overview of IPython's features (this screen).
  object?   -> Details about 'object'.
  object??  -> More detailed, verbose information about 'object'.
  %quickref -> Quick reference of all IPython specific syntax and magics.
  help      -> Access Python's own help system.

If you are in terminal IPython you can quit this screen by pressing `q`.


MAIN FEATURES
-------------

* Access to the standard Python help with object docstrings and the Python
  manuals. Simply type 'help' (no quotes) to invoke it.

* Ma

In [37]:
# ▪ 5 suggestions user-based
items_to_be_recommend[:5]['Items'].to_list()

['Black-forest-cake', 'Apple pie', 'Soup', 'Crisps', 'Salad']

In [38]:
# ▪ 5 suggestions item-based
# item_name = df["Items"].unique()
item_name_col = user_item_df[item_name]
moveis_from_item_based = user_item_df.corrwith(item_name_col).sort_values(ascending=False)
mask = moveis_from_item_based.index != item_name
moveis_from_item_based = moveis_from_item_based[mask]
moveis_from_item_based[0:5].index.to_list()
moveis_from_item_based.head()

d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Items
Bread       1.0
Cake        1.0
Frittata    1.0
Pastry      1.0
Soup        1.0
dtype: float64

In [39]:
# import ran
recommend_list = items_to_be_recommend[:10]['Items'].to_list() + moveis_from_item_based[0:10].index.to_list() + moveis_from_item_based[0:10].index.to_list()
recommend_list = list(set(recommend_list))[:10]
recommend_list = sorted(recommend_list, key=lambda x: random.random())
recommend_list

['Frittata',
 'Tea',
 'Medialuna',
 'Soup',
 'Black-forest-cake',
 'Bread',
 'Scandinavian',
 'Chicken Stew',
 'Bread Pudding',
 'Pastry']

In [40]:
recommend("Flan",random_user, df)

d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
d:\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


['Scandinavian',
 'Almond',
 'Pecan',
 'Bread',
 'Soup',
 'Coffee',
 'Black-forest-cake',
 'Bread Pudding',
 'Chicken Stew',
 'Frittata']